In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Lecture des données

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
t = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
t.head()

### Interprétation des paramètres

Pregnancies = nombre de grossesse - Glucose = taux de glucose - BloodPressure = Pression dans le sang - SkinThickness = Epaisseur de la peau - Insulin = insulin- BMI = IMC - DiabetesPedigreeFunction = l'hérédité - Age = age - Outcome = Résultat (0= pas de diabète, 1 = diabète)

In [ ]:
display(t.info(),t.head())

## Conditionnement des données

In [ ]:
t.count()

### Test si une valeur est VaN avec Insulin

In [ ]:
t[np.isnan(t.Insulin)]

## Données manquante

La fonction fillna permet de compléter simplement les paramètres manquants.

### Glucose

In [ ]:
diabetes = t.fillna(value = {'Glucose':t.Glucose.mean()})

In [ ]:
plt.hist(diabetes.Glucose, bins=80)

In [ ]:
diabetes.count()

### Bloodpressure

In [ ]:
diabetes1 = diabetes.fillna(value = {'BloodPressure':t.BloodPressure.mean()})

In [ ]:
plt.hist(diabetes1.BloodPressure, bins=80)

In [ ]:
diabetes1.count()

### SkinThickness

In [ ]:
diabetes2 = diabetes1.fillna(value = {'SkinThickness':t.SkinThickness.mean()})

In [ ]:
plt.hist(diabetes2.SkinThickness, bins=80)

In [ ]:
diabetes2.count()

### Insulin

In [ ]:
diabetes3 = diabetes2.fillna(value = {'Insulin':t.Insulin.mean()})

In [ ]:
plt.hist(diabetes3.Insulin, bins=80)

In [ ]:
diabetes3.count()

### BMI

In [ ]:
diabetes4 = diabetes3.fillna(value = {'BMI':t.BMI.mean()})

In [ ]:
plt.hist(diabetes4.BMI, bins=80)

In [ ]:
diabetes4.count()

## Déséquilibre des distributions

Certaines distributions sont déséquilibrées, et éloignées d'une loi normale :

In [ ]:
sns.distplot(diabetes4.Insulin, color='blue')

Dans ce cas, une transformation log peut améliorer l'équilibre :

In [ ]:
diabetes4['log_Insulin'] = np.log(diabetes4.Insulin+1)

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')

## Mise à l'échelle des données quantitatives

In [ ]:
diabetes4[['Age','log_Insulin']].describe()

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')
sns.kdeplot(diabetes4.Age, color='red')

Prétraitement des données

In [ ]:
from sklearn import preprocessing

Normaliser les valeurs min et à max (valeurs ramenées entre 0 et 1)

In [ ]:
minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
diabetes4[['Age', 'log_Insulin']] = minmax.fit_transform(diabetes4[['Age', 'log_Insulin']])

In [ ]:
sns.distplot(diabetes4.log_Insulin, color='blue')
sns.distplot(diabetes4.Age, color='red')

StandardScaler pour ramener la moyenne à 0 et l'écart type à 1

In [ ]:
scaler = preprocessing.StandardScaler()
diabetes4[['Age', 'log_Insulin']] = scaler.fit_transform(diabetes4[['Age', 'log_Insulin']])

In [ ]:
sns.kdeplot(diabetes4.log_Insulin, color='blue')
sns.kdeplot(diabetes4.Age, color='red')

## Encodage binaire des données qualitatives (one hot encoding)

In [ ]:
diabetes4.info()

## Création des jeux d'apprentissage et de test

Créer les jeux d'apprentissage et de test

In [ ]:
X = diabetes4.drop(['Outcome'], axis=1)
y = diabetes4.Outcome

On sépare le dataset en deux parties

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

Régression logistique pour classifier sur l'ensemble de test

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

## Mesures de performance

Importation des méthodes de mesure de performances

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

Matrice de confusion :

In [ ]:
print(confusion_matrix(y_test,y_lr))

Mesure le nombre de bonnes prédictions sur le nombre total d'observations

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

predict_proba donne un tableau de couples de probabilités :[probabilité de prédiction 0, probabilité de prédiction 1]

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

On met les probabilités de prédiction de la valeur 1 dans un dataframe

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

On affiche la distribution des probabilités de prédiction de 1, et celle des non probabilités de prédiction de 0 :

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        
plt.plot([0,0,1],[0,1,1],'g:')     
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

## Ajustement des hyperparamètres (Random Forests)

On teste les forêts aléatoires

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

validation_curve permet de tracer la courbe du score sur un ensemble d'apprentissage et sur un ensemble de test (cross validation), en faisant varier un paramètre, par exemple n_estimators

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

La méthode GridSearchCV permet de tester plusieurs combinaisons de paramètres (listés dans une grille de paramètres) et de sélectionner celle qui donne la meilleure pertinence

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

On lance l'entrainement

In [ ]:
rf_gs.fit(X_train, y_train)

Voir les paramètres sélectionnés et le score

In [ ]:
print(rf_gs.best_params_)

Sélectionne le meilleur estimateur

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

## Importance des caractéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

## XGBoost

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))